# 🚀 HyperSloth Demo Training Notebook

This notebook demonstrates how to fine-tune large language models using HyperSloth's multi-GPU capabilities. It's equivalent to running:

```bash
hypersloth-train examples/example_sharegpt_lora_2gpus.py
```

## What This Demo Does

- **Multi-GPU Training**: Uses 2 GPUs with NCCL synchronization
- **Adaptive Batching**: Optimizes sequence sorting and padding
- **LoRA Fine-tuning**: Efficient parameter updates with Low-Rank Adaptation
- **Response-only Loss**: Calculates loss only on assistant responses

## Prerequisites

1. HyperSloth installed: `pip install git+https://github.com/anhvth/HyperSloth.git`
2. At least 2 GPUs available (adjust `gpus=[0, 1]` if needed)
3. Sufficient VRAM (reduce batch size if needed)

In [ ]:
# Import HyperSloth configuration classes
from HyperSloth.hypersloth_config import *
from HyperSloth.scripts.hp_trainer import run_multiprocess_training, setup_envs

# Check GPU availability
import torch
print(f'🔥 CUDA Available: {torch.cuda.is_available()}')
print(f'🔥 GPU Count: {torch.cuda.device_count()}')
for i in range(torch.cuda.device_count()):
    print(f'   GPU {i}: {torch.cuda.get_device_name(i)}')

## ⚙️ Configuration Setup

HyperSloth uses Pydantic models for type-safe configuration. We'll set up:

1. **Data Configuration**: Dataset and tokenization settings
2. **Training Configuration**: GPU allocation and loss calculation
3. **Model Configuration**: Base model and LoRA parameters
4. **Training Arguments**: Learning rate, batch size, and optimization settings

In [ ]:
from HyperSloth.hypersloth_config import *
from HyperSloth.scripts.hp_trainer import run_multiprocess_training, setup_envs

# Main configuration using Pydantic models
hyper_config_model = HyperConfig(
    data=HFDatasetConfig(
        dataset_name="llamafactory/OpenThoughts-114k",
        split="train",
        tokenizer_name="Qwen/Qwen3-8B",  # does not matter same family qwen3
        num_samples=1000,
        instruction_part="<|im_start|>user\n",
        response_part="<|im_start|>assistant\n",
        chat_template="chatml",
    ),
    training=TrainingConfig(
        gpus=[0, 1],
        loss_type="response_only",
    ),
    fast_model_args=FastModelArgs(
        model_name="unsloth/Qwen3-0.6b-bnb-4bit",
        max_seq_length=32_000,
        load_in_4bit=True,
    ),
    lora_args=LoraArgs(
        r=8,
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_dropout=0,
        bias="none",
        use_rslora=False,
    ),
)

# Training arguments using Pydantic model
training_config_model = TrainingArgsConfig(
    output_dir="outputs/qwen3-8b-openthought-2gpus/",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    logging_steps=3,
    num_train_epochs=3,
    lr_scheduler_type="linear",
    warmup_steps=5,
    save_total_limit=2,
    weight_decay=0.01,
    optim="adamw_8bit",
    seed=3407,
    report_to="none",  # tensorboard or wawndb
)

## 🏋️ Training Arguments

Configure the training hyperparameters for optimal performance:

In [ ]:

setup_envs(hyper_config_model, training_config_model)

run_multiprocess_training(
    hyper_config_model.training.gpus, hyper_config_model, training_config_model
)

Global batch size: 16
[MP] Running on 2 GPUs


03:05:41 | INFO     | GPU1 | hp_trainer.py:44 | 🔧 GPU 1 (Rank 1/1) | Model: unsloth/Qwen3-0.6b-bnb-4bit
03:05:41 | INFO     | GPU1 | hp_trainer.py:50 | Training on GPU 1 with output_dir outputs/qwen3-8b-openthought-2gpus/
03:05:41 | INFO     | GPU1 | hp_trainer.py:53 | 🚀 Starting total training timer
03:05:41 | INFO     | GPU0 | hp_trainer.py:44 | 🔧 GPU 0 (Rank 0/1) | Model: unsloth/Qwen3-0.6b-bnb-4bit
03:05:41 | INFO     | GPU0 | hp_trainer.py:50 | Training on GPU 0 with output_dir outputs/qwen3-8b-openthought-2gpus/
03:05:41 | INFO     | GPU0 | hp_trainer.py:53 | 🚀 Starting total training timer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfl

03:06:24 | INFO     | GPU1 | logging_config.py:407 | ⏱️  model_loading: 15.26s
03:06:24 | INFO     | GPU1 | nccl_grad_sync.py:127 | [GPU=1] NCCL env: RANK=1, WORLD_SIZE=2, MASTER_ADDR=127.0.0.1, MASTER_PORT=29500
03:06:24 | INFO     | GPU1 | nccl_grad_sync.py:146 | [GPU=1] Setting current CUDA device to:0
03:06:25 | INFO     | GPU0 | logging_config.py:407 | ⏱️  model_loading: 14.46s
03:06:25 | INFO     | GPU0 | nccl_grad_sync.py:127 | [GPU=0] NCCL env: RANK=0, WORLD_SIZE=2, MASTER_ADDR=127.0.0.1, MASTER_PORT=29500
03:06:25 | INFO     | GPU0 | nccl_grad_sync.py:146 | [GPU=0] Setting current CUDA device to:0
03:06:26 | INFO     | GPU1 | nccl_grad_sync.py:160 | [GPU=1] NCCL setup complete: rank=1, world_size=2, attempt=1
03:06:26 | INFO     | GPU1 | init_modules.py:49 | Model loaded on device cuda:0, tokenizer: Qwen2TokenizerFast
03:06:26 | INFO     | GPU0 | nccl_grad_sync.py:160 | [GPU=0] NCCL setup complete: rank=0, world_size=2, attempt=1
03:06:26 | INFO     | GPU0 | init_modules.py:49

Unsloth: Making `model.base_model.model.model` require gradients
Unsloth: Making `model.base_model.model.model` require gradients


...

Step,Training Loss


03:07:31 | INFO     | GPU0 | inner_training_loop.py:100 | 
🚀 HyperSloth Token Efficiency Report (Rank 0)
+---------------------+----------------+-----------------------+
| Stage               |   Total Tokens |   Token Utilization % |
+=====================+================+=======================+
| Before Optimization |      1,283,882 |                67.39% |
+---------------------+----------------+-----------------------+
| After Optimization  |        865,203 |               100.00% |
+---------------------+----------------+-----------------------+

03:07:31 | INFO     | GPU1 | inner_training_loop.py:100 | 
🚀 HyperSloth Token Efficiency Report (Rank 1)
+---------------------+----------------+-----------------------+
| Stage               |   Total Tokens |   Token Utilization % |
+=====================+================+=======================+
| Before Optimization |      1,283,882 |                67.39% |
+---------------------+----------------+-----------------------+
| After O

KeyboardInterrupt: 

  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/unsloth/models/_utils.py", line 1043, in _unsloth_pre_compute_loss
    outputs = self._old_compute_loss(model, inputs, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/transformers/trainer.py", line 3810, in compute_loss
    outputs = model(**inputs)
              ^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/a

  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/accelerate/utils/operations.py", line 806, in __call__
    return convert_to_fp32(self.model_forward(*args, **kwargs))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/amp/autocast_mode.py", line 44, in decorate_autocast
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/peft/peft_model.py", line 1757, in forward
    return self.base_model(
           ^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/anhvth5/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1762, in _call_